<a href="https://colab.research.google.com/github/Mahmuda-104/Thesis/blob/main/CNN_New3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
from scipy.io import loadmat

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/My Drive/Data'

In [5]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 50  
fs_original = 500
fs_new = 200

control_data = []
control_path = os.path.join(data_path, 'Control')
for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []
for control_data_path in control_data:
    n_epochs = 25 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    control_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{control_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        #filtered_data=mat_data1
        
        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        control_arr.append(np.array(downsampled_data))
        
    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print( Control30[0].shape)


(25, 60, 800)


In [6]:
print("Filtered and downsampled data shape:", Control30.shape)

Filtered and downsampled data shape: (32, 25, 60, 800)


In [7]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 50  
fs_original = 500
fs_new = 200  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed49 = []
for concussed_data_path in concussed_data:
    n_epochs = 25 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    concussed_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        # filtered_data=mat_data1
        
        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
    Concussed49.append(np.array(concussed_arr))

Concussed49 = np.array(Concussed49)
print( Concussed49[0].shape)


(25, 60, 800)


In [8]:
print("Filtered and downsampled data shape:", Concussed49.shape)

Filtered and downsampled data shape: (52, 25, 60, 800)


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
dataset = np.concatenate((Control30, Concussed49), axis=0)

#(0 for control, 1 for concussed)
labels = np.concatenate((np.zeros(len(Control30)), np.ones(len(Concussed49))))

In [10]:
train_ratio = 0.8  # 80% for training
val_ratio = 0.1   # 10% for validation
test_ratio = 0.1  # 10% for testing

In [11]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

In [12]:
# import tensorflow as tf
# from tensorflow.keras import layers

# # Normalize the input data (if required)
# # X_train = X_train / 255.0
# # X_val = X_val / 255.0
# # X_test = X_test / 255.0

# # Reshape the input data based on the expected input shape of the model
# input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
# X_train = X_train.reshape((-1,) + input_shape)
# X_val = X_val.reshape((-1,) + input_shape)
# X_test = X_test.reshape((-1,) + input_shape)

# # Build the CNN model
# model = tf.keras.Sequential()
# model.add(layers.Conv2D(16, kernel_size=(6, 6),strides=2, activation='relu',padding='same', input_shape=input_shape))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Dropout(0.5))
# model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1,padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Dropout(0.5))
# model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1,padding='same'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Dropout(0.5))
# model.add(layers.Flatten())
# model.add(layers.Dense(256, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(2, activation='softmax'))  
# from keras.callbacks import EarlyStopping


# callback_list = EarlyStopping(monitor = 'val_loss',
#                               patience = 20,
#                               restore_best_weights=True)


# learning_rate = 0.0001  # Specify your desired learning rate
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=32)

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# val_accuracy=model.evaluate(X_val,y_val)

# # Print the test accuracy
# print("Test Accuracy:", test_accuracy)
# #print("Validation Accuracy", val_accuracy)

In [17]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers


input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Define a function that creates your CNN model
def create_model(learning_rate=0.005, dropout_rate=0.5):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(16, kernel_size=(6, 6), strides=2, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(2, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Create the KerasClassifier with your model function
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters to tune
parameters = {
    'learning_rate': [0.001, 0.005, 0.01],
    'dropout_rate': [0.3, 0.5, 0.7],
    'batch_size': [16, 32, 64]
}

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=parameters, cv=3)
grid_search_result = grid_search.fit(X_train, y_train)

# Print the best parameters and best accuracy
print("Best Parameters:", grid_search_result.best_params_)
#print("Best Accuracy:", grid_search_result.best_score_)


<ipython-input-17-cac2ceaec1a7>:34: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Best Parameters: {'batch_size': 16, 'dropout_rate': 0.3, 'learning_rate': 0.001}


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Split the dataset into training, validation, and test sets
# X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# Normalize the input data (if required)
# X_train = X_train / 255.0
# X_val = X_val / 255.0
# X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(16, kernel_size=(6, 6),strides=2, activation='relu',padding='same', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model with the desired loss function and optimizer
learning_rate = 0.05  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split

# # Split the dataset into training, validation, and test sets
# X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# # Normalize the input data (if required)
# # X_train = X_train / 255.0
# # X_val = X_val / 255.0
# # X_test = X_test / 255.0

# # Reshape the input data based on the expected input shape of the model
# input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
# X_train = X_train.reshape((-1,) + input_shape)
# X_val = X_val.reshape((-1,) + input_shape)
# X_test = X_test.reshape((-1,) + input_shape)

# # Build the CNN model
# model = tf.keras.Sequential()
# model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(1, activation='linear'))

# # Compile the model with the desired loss function and optimizer
# learning_rate = 0.05  # Specify your desired learning rate
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# # Train the model
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test)

# # Print the test accuracy
# print("Test Accuracy:", test_accuracy)
